<a href="https://colab.research.google.com/github/GregoryLavor/analise_lotacao/blob/main/Analise_Lotacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Análise de Dados utilizando PostgreSQL e Python (Pandas e PandaSQL)

####Configurando o setup

In [ ]:
!pip install pandasql

####Importando as bibliotecas

In [2]:
import pandas as pd
import pandasql as ps

####Lendo as bases de dados

In [3]:
tb_cargo = pd.read_csv ('https://raw.githubusercontent.com/GregoryLavor/analise_lotacao/main/Cargo.csv')
tb_departamento = pd.read_csv ('https://raw.githubusercontent.com/GregoryLavor/analise_lotacao/main/Departamento.csv')
tb_funcionario = pd.read_csv ('https://raw.githubusercontent.com/GregoryLavor/analise_lotacao/main/Funcionario.csv')
tb_lotacao = pd.read_csv ('https://raw.githubusercontent.com/GregoryLavor/analise_lotacao/main/Lotacao.csv')

#### Problema de negócio

#### 1. Listar nome (funcionário e cargo) por departamento com salário e pretensão salarial.

In [ ]:
query = """
select
cf.nome,
cc.nome as cargo,
cd.nome as departamento,
cf.pretensao_salarial,
lt.salario
from tb_lotacao as lt
inner join tb_funcionario as cf on cf.id = lt.id_funcionario
inner join tb_cargo as cc on cc.id = lt.id_cargo
inner join tb_departamento as cd on cd.id = lt.id_departamento
order by departamento,cargo asc
"""

ps.sqldf(query, locals())

,nome,cargo,departamento,pretensao_salarial,salario
0,Laura,Coordenador de Área,Administrativo,5400.0,6500.0
1,Juvencio,Coordenador de Área,Administrativo,6250.0,6500.0
2,Julia,Gestor Financeiro,Administrativo,9000.0,7000.0
3,Lucio,Gestor Financeiro,Administrativo,6000.0,7000.0
4,João,Programador,Administrativo,6000.0,7500.0
5,Antonio,Programador,Administrativo,6500.0,7500.0
6,Maria,Analista Contábil,Tecnologia da Informação,6500.0,5000.0
7,Lia,Analista Contábil,Tecnologia da Informação,5600.0,5000.0
8,Pedro,Analista de Dados,Tecnologia da Informação,5500.0,9000.0
9,Afonso,Analista de Dados,Tecnologia da Informação,5000.0,9000.0


#### 2. Listar os funcionários (nome e salário) que não sejam PCD e que possuam um salário maior que R$ 4.000,00 ordenados pelo maior salário.

In [ ]:
print('Tabela Lotação')
display(tb_lotacao.head(2))
print('Tabela Funcionário')
display(tb_funcionario.head(2))

Tabela Lotação


,id,id_funcionario,id_departamento,id_cargo,salario,cadastro,ativo
0,1,1,1,1,9000.0,2022-09-01,True
1,2,2,2,2,7500.0,2022-09-02,True


Tabela Funcionário


,id,nome,nascimento,pretensao_salarial,pcd
0,1,Pedro,1984-05-29,5500.0,True
1,2,João,1980-02-01,6000.0,False


In [8]:
query = """
select
cf.nome,
case when cf.pcd = 0 then 'nao' else 'sim' end as pcd,
lt.salario
from tb_lotacao as lt
inner join tb_funcionario as cf on cf.id = lt.id_funcionario
where cf.pcd = False and lt.salario > 4000
order by salario desc
"""

ps.sqldf(query, locals())

,nome,pcd,salario
0,Morgana,nao,8000.0
1,Dorival,nao,8000.0
2,João,nao,7500.0
3,Antonio,nao,7500.0
4,Julia,nao,7000.0
5,Lucio,nao,7000.0
6,Laura,nao,6500.0
7,Maria,nao,5000.0
8,Lia,nao,5000.0


#### 3. Listar os cargos e suas respectivas médias salariais maiores que R$ 5.000,00

In [ ]:
print('Tabela Cargo')
display(tb_cargo.head(2))
print('Tabela Lotação')
display(tb_lotacao.head(2))

Tabela Cargo


,id,nome
0,1,Analista de Dados
1,2,Programador


Tabela Lotação


,id,id_funcionario,id_departamento,id_cargo,salario,cadastro,ativo
0,1,1,1,1,9000.0,2022-09-01,True
1,2,2,2,2,7500.0,2022-09-02,True


In [ ]:
query = """
select
cc.nome as cargo,
avg(lt.salario) as media_salario
from tb_lotacao as lt
inner join tb_cargo as cc on cc.id = lt.id_cargo
group by
cc.nome
having media_salario > 5000
"""
ps.sqldf(query,locals())

,cargo,media_salario
0,Analista de Dados,9000.0
1,Analista de Infra,8000.0
2,Coordenador de Área,6500.0
3,Gestor Financeiro,7000.0
4,Programador,7500.0


#### 4. Listar os funcionários com salário e pretensão salarial, ordenando por quem recebe melhor em relação ao pretendido.

In [9]:
print('Tabela Funcionário')
display(tb_funcionario.head(2))
print('Tabela Lotação')
display(tb_lotacao.head(2))

Tabela Funcionário


,id,nome,nascimento,pretensao_salarial,pcd
0,1,Pedro,1984-05-29,5500.0,True
1,2,João,1980-02-01,6000.0,False


Tabela Lotação


,id,id_funcionario,id_departamento,id_cargo,salario,cadastro,ativo
0,1,1,1,1,9000.0,2022-09-01,True
1,2,2,2,2,7500.0,2022-09-02,True


In [16]:
query = """
select
cf.nome,
cf.pretensao_salarial,
lt.salario,
case when salario > pretensao_salarial then 'Acima' else (case when salario = pretensao_salarial then 'Igual' else 'Abaixo' end) end as relacao
from tb_lotacao as lt
inner join tb_funcionario as cf on cf.id = lt.id_funcionario
order by salario - pretensao_salarial desc
"""
ps.sqldf(query,locals())

,nome,pretensao_salarial,salario,relacao
0,Dorival,3500.0,8000.0,Acima
1,Afonso,5000.0,9000.0,Acima
2,Pedro,5500.0,9000.0,Acima
3,Morgana,5400.0,8000.0,Acima
4,João,6000.0,7500.0,Acima
5,Laura,5400.0,6500.0,Acima
6,Antonio,6500.0,7500.0,Acima
7,Lucio,6000.0,7000.0,Acima
8,Juvencio,6250.0,6500.0,Acima
9,Arthur,1100.0,1100.0,Igual
